Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 28, 28) (20000,)
Validation set (1000, 28, 28) (1000,)
Test set (1000, 28, 28) (1000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 784) (20000, 10)
Validation set (1000, 784) (1000, 10)
Test set (1000, 784) (1000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    
    #Input data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_test_dataset = tf.constant(test_dataset)
    tf_valid_dataset = tf.constant(valid_dataset)
    beta = tf.placeholder(tf.float32)
    
    #Variables
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    #Training computation
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + beta * tf.nn.l2_loss(weights)
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta : 1e-3}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 17.137951
Minibatch accuracy: 12.5%
Validation accuracy: 14.9%
Minibatch loss at step 500: 2.935423
Minibatch accuracy: 75.0%
Validation accuracy: 78.3%
Minibatch loss at step 1000: 1.780764
Minibatch accuracy: 78.1%
Validation accuracy: 79.4%
Minibatch loss at step 1500: 0.922949
Minibatch accuracy: 88.3%
Validation accuracy: 80.0%
Minibatch loss at step 2000: 0.844015
Minibatch accuracy: 84.4%
Validation accuracy: 81.4%
Minibatch loss at step 2500: 0.772983
Minibatch accuracy: 84.4%
Validation accuracy: 81.8%
Minibatch loss at step 3000: 0.676324
Minibatch accuracy: 84.4%
Validation accuracy: 82.2%
Test accuracy: 86.4%


In [7]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta = tf.placeholder(tf.float32)
  
    # Variables
    weights1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_nodes]))
    biases1 = tf.Variable(tf.zeros([hidden_nodes]))
    weights2 = tf.Variable(
        tf.truncated_normal([hidden_nodes, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))

    # Training computation
    relu_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)  
    logits = tf.matmul(relu_train, weights2) + biases2
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
  
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    relu_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
    valid_prediction = tf.nn.softmax(tf.matmul(relu_valid, weights2) + biases2)
    relu_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    test_prediction = tf.nn.softmax(tf.matmul(relu_test, weights2) + biases2)

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta : 1e-3}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 634.985107
Minibatch accuracy: 7.0%
Validation accuracy: 38.4%
Minibatch loss at step 500: 188.385284
Minibatch accuracy: 93.0%
Validation accuracy: 81.1%
Minibatch loss at step 1000: 114.422935
Minibatch accuracy: 93.8%
Validation accuracy: 83.4%
Minibatch loss at step 1500: 69.277046
Minibatch accuracy: 96.9%
Validation accuracy: 82.7%
Minibatch loss at step 2000: 41.963547
Minibatch accuracy: 100.0%
Validation accuracy: 84.7%
Minibatch loss at step 2500: 25.522366
Minibatch accuracy: 98.4%
Validation accuracy: 84.8%
Minibatch loss at step 3000: 15.550801
Minibatch accuracy: 99.2%
Validation accuracy: 85.4%
Test accuracy: 91.3%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [9]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta = tf.placeholder(tf.float32)
  
    # Variables
    weights1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_nodes]))
    biases1 = tf.Variable(tf.zeros([hidden_nodes]))
    weights2 = tf.Variable(
        tf.truncated_normal([hidden_nodes, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))

    # Training computation
    relu_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)  
    logits = tf.matmul(relu_train, weights2) + biases2
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
  
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    relu_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
    valid_prediction = tf.nn.softmax(tf.matmul(relu_valid, weights2) + biases2)
    relu_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    test_prediction = tf.nn.softmax(tf.matmul(relu_test, weights2) + biases2)

In [10]:
num_steps = 101
num_batches = 3

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = step % num_batches
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta : 1e-3}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 2 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 670.479919
Minibatch accuracy: 13.3%
Validation accuracy: 41.2%
Minibatch loss at step 2: 1206.991821
Minibatch accuracy: 41.4%
Validation accuracy: 33.2%
Minibatch loss at step 4: 480.035645
Minibatch accuracy: 81.2%
Validation accuracy: 56.2%
Minibatch loss at step 6: 314.499756
Minibatch accuracy: 98.4%
Validation accuracy: 61.7%
Minibatch loss at step 8: 312.988037
Minibatch accuracy: 99.2%
Validation accuracy: 62.9%
Minibatch loss at step 10: 311.887695
Minibatch accuracy: 100.0%
Validation accuracy: 62.9%
Minibatch loss at step 12: 311.264984
Minibatch accuracy: 100.0%
Validation accuracy: 62.9%
Minibatch loss at step 14: 310.642548
Minibatch accuracy: 100.0%
Validation accuracy: 62.9%
Minibatch loss at step 16: 310.021759
Minibatch accuracy: 100.0%
Validation accuracy: 62.9%
Minibatch loss at step 18: 309.402527
Minibatch accuracy: 100.0%
Validation accuracy: 62.9%
Minibatch loss at step 20: 308.783966
Minibatch accuracy: 100.0%
Validation a

We are getting 100% accuracy in the training sets but just 60% of accuracy in the test set because the network learn too much of the training set. There isn't generalization because of the lack of data so the test accuracy is really poor. This is what it's called overfitting.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [11]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta = tf.placeholder(tf.float32)
  
    # Variables
    weights1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_nodes]))
    biases1 = tf.Variable(tf.zeros([hidden_nodes]))
    weights2 = tf.Variable(
        tf.truncated_normal([hidden_nodes, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))

    # Training computation
    relu_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1) 
    drop = tf.nn.dropout(relu_train, 0.5)
    logits = tf.matmul(drop, weights2) + biases2
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
  
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    relu_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
    valid_prediction = tf.nn.softmax(tf.matmul(relu_valid, weights2) + biases2)
    relu_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    test_prediction = tf.nn.softmax(tf.matmul(relu_test, weights2) + biases2)

In [12]:
num_steps = 101
num_batches = 3

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = step % num_batches
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta : 1e-3}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 2 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 886.308167
Minibatch accuracy: 7.8%
Validation accuracy: 41.0%
Minibatch loss at step 2: 872.470764
Minibatch accuracy: 60.9%
Validation accuracy: 41.4%
Minibatch loss at step 4: 657.939941
Minibatch accuracy: 62.5%
Validation accuracy: 49.7%
Minibatch loss at step 6: 372.871124
Minibatch accuracy: 79.7%
Validation accuracy: 68.6%
Minibatch loss at step 8: 313.341858
Minibatch accuracy: 97.7%
Validation accuracy: 71.0%
Minibatch loss at step 10: 330.718018
Minibatch accuracy: 96.1%
Validation accuracy: 70.9%
Minibatch loss at step 12: 314.139618
Minibatch accuracy: 97.7%
Validation accuracy: 70.9%
Minibatch loss at step 14: 310.185272
Minibatch accuracy: 100.0%
Validation accuracy: 70.9%
Minibatch loss at step 16: 309.565613
Minibatch accuracy: 100.0%
Validation accuracy: 70.9%
Minibatch loss at step 18: 308.950287
Minibatch accuracy: 100.0%
Validation accuracy: 70.1%
Minibatch loss at step 20: 310.946381
Minibatch accuracy: 97.7%
Validation accura

Due to the use of Dropout we have improved our Test accuracy. This means that our network is more capable of generalize.

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [86]:
batch_size = 128
hidden_nodes = 1568
hidden_nodes2 = 100

graph = tf.Graph()
with graph.as_default():
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta = tf.placeholder(tf.float32)
    global_step = tf.Variable(0)

  
    # Variables
    weights1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_nodes], stddev=np.sqrt(2.0/(image_size*image_size))))
    biases1 = tf.Variable(tf.zeros([hidden_nodes]))
    
    weights2 = tf.Variable(
        tf.truncated_normal([hidden_nodes, hidden_nodes2], stddev=np.sqrt(2.0/hidden_nodes)))
    biases2 = tf.Variable(tf.zeros([hidden_nodes2]))
    
    weights3 = tf.Variable(
        tf.truncated_normal([hidden_nodes2, num_labels], stddev=np.sqrt(2.0/hidden_nodes2)))
    biases3 = tf.Variable(tf.zeros([num_labels]))

    # Training computation
    relu1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
    relu2_train = tf.nn.relu(tf.matmul(relu1_train, weights2) + biases2)
    drop2 = tf.nn.dropout(relu2_train, 0.5)
    logits = tf.matmul(drop2, weights3) + biases3
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights3))
  
    # Optimizer
    learning_rate = tf.train.exponential_decay(0.5, global_step, 1000, 0.6, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    
    relu1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
    relu2_valid = tf.nn.relu(tf.matmul(relu1_valid, weights2) + biases2)
    valid_prediction = tf.nn.softmax(tf.matmul(relu2_valid, weights3) + biases3)

    relu1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    relu2_test = tf.nn.relu(tf.matmul(relu1_test, weights2) + biases2)
    test_prediction = tf.nn.softmax(tf.matmul(relu2_test, weights3) + biases3)

In [87]:
num_steps = 9001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta : 1e-3}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.749169
Minibatch accuracy: 8.6%
Validation accuracy: 34.1%
Minibatch loss at step 500: 1.297678
Minibatch accuracy: 85.9%
Validation accuracy: 85.5%
Minibatch loss at step 1000: 1.070489
Minibatch accuracy: 85.9%
Validation accuracy: 86.4%
Minibatch loss at step 1500: 0.639619
Minibatch accuracy: 93.8%
Validation accuracy: 86.3%
Minibatch loss at step 2000: 0.585971
Minibatch accuracy: 96.1%
Validation accuracy: 87.8%
Minibatch loss at step 2500: 0.545958
Minibatch accuracy: 94.5%
Validation accuracy: 86.6%
Minibatch loss at step 3000: 0.513555
Minibatch accuracy: 94.5%
Validation accuracy: 86.7%
Minibatch loss at step 3500: 0.397584
Minibatch accuracy: 99.2%
Validation accuracy: 87.8%
Minibatch loss at step 4000: 0.427622
Minibatch accuracy: 96.1%
Validation accuracy: 87.6%
Minibatch loss at step 4500: 0.413916
Minibatch accuracy: 98.4%
Validation accuracy: 87.3%
Minibatch loss at step 5000: 0.434654
Minibatch accuracy: 94.5%
Validation accuracy

Lets add some more dropouts and see if we can improve the 93.8%


In [88]:
batch_size = 128
hidden_nodes = 1568
hidden_nodes2 = 100

graph = tf.Graph()
with graph.as_default():
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta = tf.placeholder(tf.float32)
    global_step = tf.Variable(0)

  
    # Variables
    weights1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_nodes], stddev=np.sqrt(2.0/(image_size*image_size))))
    biases1 = tf.Variable(tf.zeros([hidden_nodes]))
    
    weights2 = tf.Variable(
        tf.truncated_normal([hidden_nodes, hidden_nodes2], stddev=np.sqrt(2.0/hidden_nodes)))
    biases2 = tf.Variable(tf.zeros([hidden_nodes2]))
    
    weights3 = tf.Variable(
        tf.truncated_normal([hidden_nodes2, num_labels], stddev=np.sqrt(2.0/hidden_nodes2)))
    biases3 = tf.Variable(tf.zeros([num_labels]))

    # Training computation
    relu1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
    drop1 = tf.nn.dropout(relu1_train, 0.5)
    relu2_train = tf.nn.relu(tf.matmul(drop1, weights2) + biases2)
    drop2 = tf.nn.dropout(relu2_train, 0.5)
    logits = tf.matmul(drop2, weights3) + biases3
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights3))
  
    # Optimizer
    learning_rate = tf.train.exponential_decay(0.5, global_step, 1000, 0.6, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    
    relu1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
    relu2_valid = tf.nn.relu(tf.matmul(relu1_valid, weights2) + biases2)
    valid_prediction = tf.nn.softmax(tf.matmul(relu2_valid, weights3) + biases3)

    relu1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    relu2_test = tf.nn.relu(tf.matmul(relu1_test, weights2) + biases2)
    test_prediction = tf.nn.softmax(tf.matmul(relu2_test, weights3) + biases3)

In [90]:
num_steps = 9001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta : 1e-3}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.045128
Minibatch accuracy: 8.6%
Validation accuracy: 26.9%
Minibatch loss at step 500: 1.430125
Minibatch accuracy: 81.2%
Validation accuracy: 85.1%
Minibatch loss at step 1000: 1.140199
Minibatch accuracy: 82.8%
Validation accuracy: 85.6%
Minibatch loss at step 1500: 0.810304
Minibatch accuracy: 93.0%
Validation accuracy: 86.6%
Minibatch loss at step 2000: 0.726891
Minibatch accuracy: 92.2%
Validation accuracy: 86.9%
Minibatch loss at step 2500: 0.640858
Minibatch accuracy: 91.4%
Validation accuracy: 86.8%
Minibatch loss at step 3000: 0.585880
Minibatch accuracy: 90.6%
Validation accuracy: 87.0%
Minibatch loss at step 3500: 0.504804
Minibatch accuracy: 95.3%
Validation accuracy: 87.9%
Minibatch loss at step 4000: 0.631344
Minibatch accuracy: 89.1%
Validation accuracy: 87.9%
Minibatch loss at step 4500: 0.554174
Minibatch accuracy: 90.6%
Validation accuracy: 88.0%
Minibatch loss at step 5000: 0.551874
Minibatch accuracy: 90.6%
Validation accuracy

In [ ]:
With the dropouts the accuracy goes down to 92.9%